In [1]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent  
sys.path.append(str(project_root))

from src.arcgis_rest_downloader import (
    get_arcgis_services_to_pd,
    download_and_create_cog,
)

# Orthophotos

In [2]:
service_url = "https://gis.stmk.gv.at/image/rest/services/OGD_DOP"
# -> check out the REST interface with the above service_url, it is quite useful! 

# Test getting services
print("Fetching available services...")
services_df = get_arcgis_services_to_pd(service_url)
display(services_df.head(50))

Fetching available services...


,category,service_name,year,type,name
0,OGD_DOP,Falschfarben_2008_2011,2008,ImageServer,OGD_DOP/Falschfarben_2008_2011
1,OGD_DOP,Falschfarben_2011_2013,2011,ImageServer,OGD_DOP/Falschfarben_2011_2013
2,OGD_DOP,Falschfarben_2013_2015,2013,ImageServer,OGD_DOP/Falschfarben_2013_2015
3,OGD_DOP,Falschfarben_2016_2018,2016,ImageServer,OGD_DOP/Falschfarben_2016_2018
4,OGD_DOP,Falschfarben_2019_2021,2019,ImageServer,OGD_DOP/Falschfarben_2019_2021
5,OGD_DOP,Falschfarben_2022_2024,2022,ImageServer,OGD_DOP/Falschfarben_2022_2024
6,OGD_DOP,Falschfarben_akt,NaN,ImageServer,OGD_DOP/Falschfarben_akt
7,OGD_DOP,Flug_2003_2007_RGB,2003,ImageServer,OGD_DOP/Flug_2003_2007_RGB
8,OGD_DOP,Flug_2008_2011_RGB,2008,ImageServer,OGD_DOP/Flug_2008_2011_RGB
9,OGD_DOP,Flug_2011_2013_RGB,2011,ImageServer,OGD_DOP/Flug_2011_2013_RGB


### Download all intersecting tiles for a single service name

In [3]:
from src.arcgis_rest_downloader.downloader import download_raster_tiles_from_service_url

service_name = "Falschfarben_2008_2011"

output_path=f"../data/processed/orthophotos_gis_stmk/{service_name.lower()}.tif"
bbox_gpkg_path="../data/Verschneidung_NPG_Johnsbachtal_Europaschutzgebiet_BBOX_SMALL.gpkg"
raw_data_folder=f"../data/raw/orthophoto_tiles_gis_stmk/{service_name.lower()}"


tiles = download_raster_tiles_from_service_url(
            service_url=service_url,
            service_name=service_name,
            output_directory=raw_data_folder,
            bbox_gpkg_path=bbox_gpkg_path,
            max_retry=5,
            outSRS=32633,
        )

Metadata saved to ../data/raw/orthophoto_tiles_gis_stmk/falschfarben_2008_2011/Falschfarben_2008_2011.json

 Gathering tile information...


Processing tiles: 100%|██████████| 14/14 [00:01<00:00,  8.98it/s]


Downloaded 6 tiles to ../data/raw/orthophoto_tiles_gis_stmk/falschfarben_2008_2011


### Download all intersecting tiles for a list of service names + create COGs

In [4]:
# Define which services to use (those that where available May 2025)
services = [
    "Falschfarben_2008_2011",
    # "Falschfarben_2013_2015",
    # "Falschfarben_2016_2018",
    # "Falschfarben_2019_2021",
    # "Falschfarben_2022_2024",
    # "Flug_2003_2007_RGB",
    # "Flug_2008_2011_RGB",
    # "Flug_2013_2015_RGB",
    # "Flug_2016_2018_RGB",
    # "Flug_2019_2021_RGB",
    # "Flug_2022_2024_RGB",
    # "SW_1994_2001",
]

services_to_download_df = services_df.loc[services_df["service_name"].isin(services)]

In [5]:
# loop through services_df and download all tiles from each service using the service_url and service_name
for index, row in services_to_download_df.iterrows():
    service_name = row["service_name"]
    print(f"Downloading tiles from service: {service_name}")

    download_and_create_cog(
        service_url=service_url,
        service_name=service_name,
        output_path=f"../data/processed/orthophotos_gis_stmk/{service_name.lower()}.tif",
        bbox_gpkg_path="../data/Verschneidung_NPG_Johnsbachtal_Europaschutzgebiet_BBOX_SMALL.gpkg",
        raw_data_folder=f"../data/raw/orthophoto_tiles_gis_stmk/{service_name.lower()}",
        out_nodata_value=0
    )

Metadata saved to ../data/raw/orthophoto_tiles_gis_stmk/falschfarben_2008_2011/Falschfarben_2008_2011.json

 Gathering tile information...


Processing tiles: 100%|██████████| 14/14 [00:02<00:00,  6.32it/s]


Downloaded 6 tiles to ../data/raw/orthophoto_tiles_gis_stmk/falschfarben_2008_2011
Creating VRT...
0...10...20...30...40...50...60...70...80...90...100 - done.
Converting to COG...
Creating output file that is 12819P x 15265L.
Processing ../data/raw/orthophoto_tiles_gis_stmk/falschfarben_2008_2011.vrt [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12819, 15265
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12819, 15265
0...10...20...30...40...50...60...70.

KeyboardInterrupt: 

# TODO

## Elevation Data

In [5]:
#TODO: Enable elevation data procssing, or just skip processing and leave singles tiles in disk? @Daniel

service_url = "https://gis.stmk.gv.at/image/rest/services/OGD_Hoehen"

# Test getting services
print("Fetching available services...")
services_df = get_arcgis_services_to_pd(service_url)
display(services_df.head(50))

Fetching available services...


,category,service_name,year,type,name
0,OGD_Hoehen,ALS_Hoehen_2008_2014_DGM_1m_UTM33N_32633_stmkt...,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DGM_1m_UTM33N_...
1,OGD_Hoehen,ALS_Hoehen_2008_2014_DGM_50cm_GKM31_31255,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DGM_50cm_GKM31...
2,OGD_Hoehen,ALS_Hoehen_2008_2014_DGM_50cm_UTM33N_32633,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DGM_50cm_UTM33...
3,OGD_Hoehen,ALS_Hoehen_2008_2014_DOM_1m_UTM33N_32633_stmkt...,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DOM_1m_UTM33N_...
4,OGD_Hoehen,ALS_Hoehen_2008_2014_DOM_50cm_GKM31_31255,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DOM_50cm_GKM31...
5,OGD_Hoehen,ALS_Hoehen_2008_2014_DOM_50cm_UTM33N_32633,2008,ImageServer,OGD_Hoehen/ALS_Hoehen_2008_2014_DOM_50cm_UTM33...
6,OGD_Hoehen,ALS_Hoehen_2022_2027_DGM_50cm_GKM31_31255,2022,ImageServer,OGD_Hoehen/ALS_Hoehen_2022_2027_DGM_50cm_GKM31...
7,OGD_Hoehen,ALS_Hoehen_2022_2027_DGM_50cm_GKM34_31256,2022,ImageServer,OGD_Hoehen/ALS_Hoehen_2022_2027_DGM_50cm_GKM34...
8,OGD_Hoehen,ALS_Hoehen_2022_2027_DGM_50cm_UTM33N_32633,2022,ImageServer,OGD_Hoehen/ALS_Hoehen_2022_2027_DGM_50cm_UTM33...
9,OGD_Hoehen,ALS_Hoehen_2022_2027_DOM_50cm_GKM31_31255,2022,ImageServer,OGD_Hoehen/ALS_Hoehen_2022_2027_DOM_50cm_GKM31...


In [6]:
# Define which services to use
services = [
    "ALS_Hoehen_2008_2014_DGM_1m_UTM33N_32633_stmktrafo",
    "ALS_Hoehen_2008_2014_DOM_1m_UTM33N_32633_stmktrafo",
]

services_to_download_df = services_df.loc[services_df["service_name"].isin(services)]

In [ ]:
# loop through services_df and download all tiles from each service using the service_url and service_name
for index, row in services_to_download_df.iterrows():
    service_name = row["service_name"]
    print(f"Downloading tiles from service: {row['service_name']}")

    download_and_create_cog(
        service_url=service_url,
        service_name=row["service_name"],
        category="elevation",
        output_path=f"../data/processed/elevation_gis_stmk/{service_name.lower()}.tif",
        bbox_gpkg_path="../data/raw/Verschneidung_NPG_Johnsbachtal_Europaschutzgebiet_BBOX.gpkg",
        raw_data_folder=f"../data/raw/elevation_tiles_gis_stmk/{service_name.lower()}",
        out_nodata_value=-9999
    )

## Convert Local Elevation Data

In [ ]:
from src.data.processing.processing_utils import create_vrt_from_tiles, create_cog

for type in ["dtm", "dsm", "ndsm"]:
    tiles = list(Path(f"../data/raw/elevation_tiles_waldstmk/{type}").iterdir())
    vrt_path = Path(f"../data/raw/elevation_tiles_waldstmk/{type}.vrt")
    output_path = f"../data/processed/elevation_waldstmk/{type}.tif"

    # Create VRT
    print("Creating VRT...")
    create_vrt_from_tiles(tiles, vrt_path)

    # Convert to COG
    print("Converting to COG...")
    create_cog(vrt_path, output_path, category="elevation", outSRS=32633, out_nodata_value=-9999)

## DEM Processing

In [ ]:
from src.data.processing.processing_utils import dem_processing

dem_processing(
    input_path=Path("D:/Nextcloud/HabitAlp2.0/Originaldaten/processed/elevation_waldstmk/dtm.tif"),
    output_folder=Path("../data/processed"),
)